In [1]:
#TO DO

# - Create correct directories in get_y
# - Do I want to make the padding value 0 so i ignore all rows with 0??
# why do I get loss nan??
# I have nans in my X_train /X_test
# should i use a different optimizer for my compiler?? maybe rmsprop

In [27]:
import pandas as pd
import os
import csv
import numpy as np
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
!pwd

/Users/hogangardner/code/roalko/squatpro/notebooks


In [28]:
ROOT_DATA_DIR = '../raw_data/CSV_Features'
DATA_SUB_DIRs = ["Train", "Test"]
CLS_LIST = ["Bad", "Good"]

def get_y(data_path, class_list):
    
    data_dict = {"csv_paths":[], "csv_files":[], "labels":[]}
    
    for i, clss in enumerate(class_list):
        dir_path = os.path.join(data_path, clss)
        list_csvs = [l for l in os.listdir(dir_path)]
        print(f"Found {len(list_csvs)} CSVs in {dir_path} Directory")
        data_dict["csv_files"].extend(list_csvs)
        data_dict["labels"].extend([i for k in range(len(list_csvs))])
        data_dict["csv_paths"].extend([dir_path for _ in range(len(list_csvs))])
    
    total_csvs = len(data_dict["csv_files"])
    total_labels = len(data_dict["labels"])
    print(f"Number of csvs : {total_csvs}")
    print(f"Number of Labels : {total_labels}")
    return data_dict

train_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[0]),
                               class_list=CLS_LIST))


test_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[1]),
                               class_list=CLS_LIST)) 

Found 40 CSVs in ../raw_data/CSV_Features/Train/Bad Directory
Found 40 CSVs in ../raw_data/CSV_Features/Train/Good Directory
Number of csvs : 80
Number of Labels : 80
Found 10 CSVs in ../raw_data/CSV_Features/Test/Bad Directory
Found 10 CSVs in ../raw_data/CSV_Features/Test/Good Directory
Number of csvs : 20
Number of Labels : 20


In [29]:
train_data.head()

,csv_paths,csv_files,labels
0,../raw_data/CSV_Features/Train/Bad,68.csv,0
1,../raw_data/CSV_Features/Train/Bad,54.csv,0
2,../raw_data/CSV_Features/Train/Bad,41.csv,0
3,../raw_data/CSV_Features/Train/Bad,55.csv,0
4,../raw_data/CSV_Features/Train/Bad,69.csv,0


In [30]:
train_data['csv_paths'][0]

'../raw_data/CSV_Features/Train/Bad'

In [31]:
train_data.tail()

,csv_paths,csv_files,labels
75,../raw_data/CSV_Features/Train/Good,11.csv,1
76,../raw_data/CSV_Features/Train/Good,13.csv,1
77,../raw_data/CSV_Features/Train/Good,12.csv,1
78,../raw_data/CSV_Features/Train/Good,9.csv,1
79,../raw_data/CSV_Features/Train/Good,8.csv,1


In [32]:
y_train = train_data['labels']

In [33]:
y_train

0     0
1     0
2     0
3     0
4     0
     ..
75    1
76    1
77    1
78    1
79    1
Name: labels, Length: 80, dtype: int64

In [34]:
y_test = test_data['labels']

In [35]:
def csv_to_list_of_lists(csv_path):
    df = pd.read_csv(csv_path)
    df.fillna(180, inplace=True)
    list_of_lists = df.feature_1.values.tolist()
    return list_of_lists

In [36]:
def get_x(dataframe):
  list_of_csv_files = [os.path.join(row['csv_paths'], row["csv_files"]) for index, row in dataframe.iterrows()]
  X_list = [csv_to_list_of_lists(f) for f in list_of_csv_files]
  return X_list

In [37]:
#X_train is a list of lists of different list lengths
X_train = get_x(train_data)

In [38]:
#normalizing the X_train
X_train = [[float(j)/180 for j in i] for i in X_train] 

In [39]:
#pad X_train so all arrays are of the same shape
X_train_pad = pad_sequences(X_train, dtype='float32',padding='post',value=-1000)
X_train_pad.shape

(80, 176)

In [40]:
#input data must be of shape(number of sequences, number of observations per sequence, observation size)
#add a last dimension of 1
X_train_pad = np.expand_dims(X_train_pad, 2)
X_train_pad.shape

(80, 176, 1)

In [41]:
#X_test is a list of lists of different list lengths
X_test = get_x(test_data)

In [42]:
#normalizing the X_test
X_test = [[float(j)/180 for j in i] for i in X_test] 

In [43]:
#pad X_test so all arrays are of the same shape
X_test_pad = pad_sequences(X_test, dtype='float32',padding='post',value=-1000)

#input data must be of shape(number of sequences, number of observations per sequence, observation size)
#add a last dimension of 1
X_test_pad = np.expand_dims(X_test_pad, 2)

In [44]:
print(X_train_pad.shape)
print(y_train.shape)
print(X_test_pad.shape)
print(y_test.shape)

(80, 176, 1)
(80,)
(20, 135, 1)
(20,)


In [45]:
X_test_pad = np.array(X_test_pad)
y_test = np.array(y_test)
X_train_pad = np.array(X_train_pad)
y_train = np.array(y_train)

In [46]:
X_test_pad

array([[[ 9.8420858e-01],
        [ 9.6434009e-01],
        [ 9.6434009e-01],
        ...,
        [-1.0000000e+03],
        [-1.0000000e+03],
        [-1.0000000e+03]],

       [[ 9.4678652e-01],
        [ 9.4678652e-01],
        [ 9.4678652e-01],
        ...,
        [-1.0000000e+03],
        [-1.0000000e+03],
        [-1.0000000e+03]],

       [[ 9.3531430e-01],
        [ 9.4639975e-01],
        [ 9.4639975e-01],
        ...,
        [-1.0000000e+03],
        [-1.0000000e+03],
        [-1.0000000e+03]],

       ...,

       [[ 9.1696727e-01],
        [ 9.1696727e-01],
        [ 8.9573228e-01],
        ...,
        [-1.0000000e+03],
        [-1.0000000e+03],
        [-1.0000000e+03]],

       [[ 9.6943223e-01],
        [ 9.9678171e-01],
        [ 8.7710971e-01],
        ...,
        [-1.0000000e+03],
        [-1.0000000e+03],
        [-1.0000000e+03]],

       [[ 8.5975575e-01],
        [ 9.4406283e-01],
        [ 8.4102827e-01],
        ...,
        [-1.0000000e+03],
        [-1.000

In [23]:
#dimensions look good and ready to rain 

In [54]:
#LSTM RNN Model
model_LSTM = Sequential()

##change masking layer to padded amount
model_LSTM.add(layers.Masking(mask_value=-1000))

#how many units should the first layer be??
model_LSTM.add(layers.LSTM(100, activation='tanh'))  
model_LSTM.add(layers.Dense(80, activation='relu'))
model_LSTM.add(layers.Dense(60, activation='relu'))
model_LSTM.add(layers.Dense(20, activation='tanh'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))
model_LSTM.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#More Complex LSTM RNN Model
model_LSTM = Sequential()

##change masking layer to padded amount
model_LSTM.add(layers.Masking(mask_value=-1.))

##Hogan's first layer (do i need to add an input dimension??)
model.add(layers.SimpleRNN(500, input_dim=1, activation='relu'))

#how many units should the first layer be??
model_LSTM.add(layers.LSTM(units=500, activation='tanh'))  
model_LSTM.add(layers.Dense(250, activation='relu'))
model_LSTM.add(layers.Dense(125, activation='relu'))
model_LSTM.add(layers.Dense(75, activation='relu'))
model_LSTM.add(layers.Dense(25, activation='relu'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [55]:
model_LSTM.fit(X_train_pad, y_train, 
              epochs=10, 
              batch_size=32, 
              verbose=1, 
              validation_split=0.2)

Epoch 1/10
2/2 [==============================] - 6s 2s/step - loss: 0.6912 - accuracy: 0.6146 - val_loss: 0.7330 - val_accuracy: 0.0000e+00
Epoch 2/10
2/2 [==============================] - 0s 160ms/step - loss: 0.6824 - accuracy: 0.6042 - val_loss: 0.7781 - val_accuracy: 0.0000e+00
Epoch 3/10
2/2 [==============================] - 0s 156ms/step - loss: 0.6747 - accuracy: 0.6146 - val_loss: 0.8468 - val_accuracy: 0.0000e+00
Epoch 4/10
2/2 [==============================] - 0s 151ms/step - loss: 0.6605 - accuracy: 0.6354 - val_loss: 0.9721 - val_accuracy: 0.0000e+00
Epoch 5/10
2/2 [==============================] - 0s 162ms/step - loss: 0.6549 - accuracy: 0.6354 - val_loss: 1.1259 - val_accuracy: 0.0000e+00
Epoch 6/10
2/2 [==============================] - 0s 152ms/step - loss: 0.6493 - accuracy: 0.6458 - val_loss: 1.1588 - val_accuracy: 0.0000e+00
Epoch 7/10
2/2 [==============================] - 0s 148ms/step - loss: 0.6597 - accuracy: 0.6146 - val_loss: 1.0696 - val_accuracy: 0.0000

In [56]:
#Evaluate the model on the test set
model_LSTM.evaluate(X_test_pad, y_test)

1/1 [==============================] - 2s 2s/step - loss: 0.7643 - accuracy: 0.5000


[0.7642539739608765, 0.5]

In [53]:
model_LSTM.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, None, 1)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_4 (Dense)              (None, 80)                8080      
_________________________________________________________________
dense_5 (Dense)              (None, 60)                4860      
_________________________________________________________________
dense_6 (Dense)              (None, 20)                1220      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 21        
Total params: 54,981
Trainable params: 54,981
Non-trainable params: 0
__________________________________________________

In [ ]:
#creating a grid search
#we have three models
#each one of those models has possible parameters

#SCI-KIT LEARN DOCUMENTATION

simple_RNN_parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
GRU_parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
LSTM_parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}